## Подготовка данных

### Описание данных

*Исследование надёжности заёмщиков*

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Файл data хранит следующую информацию о пользователях: 
- `children` — количество детей в семье;
- `days_employed` — общий трудовой стаж в днях;
- `dob_years` — возраст клиента в годах;
- `education` — уровень образования клиента;
- `education_id` — идентификатор уровня образования;
- `family_status` — семейное положение;
- `family_status_id` — идентификатор семейного положения;
- `gender` — пол клиента;
- `income_type` — тип занятости;
- `debt` — имел ли задолженность по возврату кредитов;
- `total_income` — ежемесячный доход;
- `purpose` — цель получения кредита.

### Загрузка данных

In [1]:
# импорт библиотеки pandas
import pandas as pd
# чтение файла с данными и сохранение в df
server_path = '/datasets/data.csv'
local_path = 'data.csv' 
#использую конструкцию try-except удобства запуска локально и через сервер практикума
try:  
    df = pd.read_csv(server_path)     
except: 
    df = pd.read_csv(local_path) 
df.head()    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Оценка объёма данных в таблице: 

In [2]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных в столбцах — int64, float64, object.

**Вывод**

В каждой строке таблицы - статистика платёжеспособность клиента. Большая часть колонок описывает клиента: количество детей в семье, общий трудовой стаж, возраст, уровень образования, семейное положение, пол, тип занятости, доход. Остальные данные сообщают о кредитной банковской истории клиента: имел ли задолженность по возврату кредитов, цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, значение '-8437.673028' в стоблце 'days_employed' не соответсвует стандарной записи общего трудового стажа в днях, а также значения в стоблце 'education' оформлены с расхождением хорошего стиля (записаны в разных регистрах). 

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала посчитаем, сколько в таблице пропущенных значений.

In [3]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Источник трафика пропущен в 2174 случаях. Это примерно 10% от всех значений датафрейма df, в котором 21525 строк. Удалив строки с NaN, мы лишим себя почти 1/10 данных, что впоследствии повлияет на метрики возврата кредита и выводы исследования.

Методом isna() найдём все строки с пропусками в столбце total_income и days_employed и просмотрим первые пять.

In [4]:
# подсчёт пропусков для столбца 'days_employed'
df[df['days_employed'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [5]:
# подсчёт пропусков для столбца 'total_income'
df[df['days_employed'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Таблицы, демонстрирующие пропуски, и их количество в столбцах 'days_employed' и 'total_income' совпадают. Значит, у клиентов, не имеющих ежемесячного дохода соответственно отсутствует информация по трудовому стажу. На первый взгляд можно сделать вывод, что причина появления пропусков в данных для этой категории клиентов: отсутствие официального места работы. Но в столбце 'income_type' полностью заполнены типы занятости клиентов. Значит возможная причина появления пропусков искажение данных при передаче или сокрытие информации.
Общий трудовой стаж в днях и ежемесячный доход — количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана.

Значения общего трудового стажа в днях представлены в некоторых случаях отрицательными, поэтому, используя метод apply(abs), приведем все значения столбца days_employed к положительным.

In [6]:
# перевод значений столбца 'days_employed' к положительным числам
df['days_employed'] = df['days_employed'].apply(abs)

Аномально большие значения общего трудового стажа категории 'пенсионер' предполагаю, что даны в часах, следовательно необходимо перевести данные в дни, поделив значения на 24. 

In [7]:
# перевод значений столбца 'days_employed' в дни
df.loc[df['days_employed'] >30000, 'days_employed'] = df.loc[df['days_employed'] >30000, 'days_employed'] / 24
# проверка выполнения кода
df[df['income_type'] == 'пенсионер'].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,16678.380705,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,14106.331371,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,15147.853723,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости


Определим уникальные значения и количество упоминайний в стобце 'income_type'.

In [8]:
# просмотр уникальных значений и их количества типов занятости
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Заполним пропущенные значения в столбце 'days_employed' средним значением общего трудового стажа для каждой категории типа занятости.

In [9]:
# перебор типов занятости в цикле и замена пропущенных значений на среднее значение трудового стажа
print('Пропуски до:', df['days_employed'].isna().sum())
for item in df['income_type'].unique():
    mean = df.loc[df['income_type'] == item, 'days_employed'].mean()
    print(item, mean)
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == item), 'days_employed'] = mean
print('Пропуски после:', df['days_employed'].isna().sum())

Пропуски до: 2174
сотрудник 2326.4992159718063
пенсионер 15208.478801869218
компаньон 2111.5243982977295
госслужащий 3399.896901695746
безработный 15267.235531008522
предприниматель 520.8480834953765
студент 578.7515535382181
в декрете 3296.7599620220594
Пропуски после: 0


In [10]:
# просмотр максимального значения в столбце 'total_income'
df['total_income'].max()

2265604.028722744

In [11]:
# просмотр минимального значения в столбце 'total_income'
df['total_income'].min()

20667.26379327158

Заполним пропущенные значения в столбце 'total_income' медианным значением ежемесячного дохода для каждой категории типа занятости.

In [12]:
# перебор типов занятости в цикле и замена пропущенных значений на медианное значение ежемесячного дохода
print('Пропуски до:', df['total_income'].isna().sum())
for item in df['income_type'].unique():
    median = df.loc[df['income_type'] == item, 'total_income'].median()
    print(item, median)
    df.loc[(df['total_income'].isna()) & (df['income_type'] == item), 'total_income'] = median
print('Пропуски после:', df['total_income'].isna().sum())

Пропуски до: 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Пропуски после: 0


**Вывод**

В процессе предобработки обнаружены пропущенные значения в столбцах 'days_employed' и 'total_income'.
Пропущенные значения заменены в столбце 'days_employed' средним значением. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, поэтому для заполнения пропущенных значений в столбце ежемесячного дохода 'total_income' применён метод median() для каждой категорий.

### Замена типа данных

Проанализируем какие представлены значения в переменных по типу, вызвав метод info()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [14]:
# перевод вещественных чисел в целые в столбце 'days_employed'
df['days_employed'] = df['days_employed'].astype('int')
# перевод вещественных чисел в целые в столбце 'total_income'
df['total_income'] = df['total_income'].astype('int')

In [15]:
# проверка выполнения метода astype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

В таблице представлены столбцы с типом данных int64, float64, object. Вещественные числа (тип данных float64) неточны и могут привести к ошибкам при арифметических операциях, а также увеличивают размер файла и время обработки данных. Поэтому, использовав метод astype, вещественные числа были переведены в целые.

### Обработка дубликатов

Выявим неявные дубликаты в столбце 'education'.

In [16]:
# просмотр уникальных значений и их количества столбца 'education'
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [17]:
# возвращание столбца 'education' с символами, приведёнными к нижнему регистру
df['education'] = df['education'].str.lower()

Подсчитаем количество дубликатов, используя метод duplicated() в сочетании с sum().

In [18]:
# подсчёт явных дубликатов
duplicated_number = df.duplicated().sum()
duplicated_number

71

In [19]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Проверим выполнение кода.

In [20]:
# проверка выполнения метода drop_duplicates
duplicated_number = df.duplicated().sum()
duplicated_number

0

**Вывод**

Основные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации. В процессе обработки дубликатов был использован метод drop_duplicates с удалением старых индексов и формированием новых.

### Лемматизация

Импортируем библиотеку pymystem3. При помощи метода unique() определим уникальные значения столбца 'purpose'. К этим значениям  применим лемматизацию и подсчитаем количество лемматизированных слов (для этого вызовем специальный контейнер Counter из модуля collections).

In [21]:
# импорт pymystem3
from pymystem3 import Mystem
m = Mystem()
# вызов специальный контейнер Counter из модуля collections
from collections import Counter

In [22]:
# установление уникальные значения столбца 'purpose', лемматизация и подсчет количества упоминаний
text = df['purpose'].unique()
separator = ' '
text = separator.join(text)
lemmas = m.lemmatize(text)
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Создадим функцию, которая будет лематизировать последовательно каждую ячейку столбца 'purpose' и проверять, какая из категорий обнаруживается в лемматизированном варианте.

In [23]:
def purpose_lemmas(purpose):
    if ('недвиж' in purpose or 'жилье' in purpose):
        return 'недвижимость'
    elif 'автомоб' in purpose:
        return 'автомобиль'
    elif 'образ' in purpose:
        return 'образование'
    elif 'свадь' in purpose:
        return 'свадьба'
    else:
        return 'цель не определена'

    
df["lemmas_purposes"] = df["purpose"]. apply(purpose_lemmas)
df_unique_purp_count = df["lemmas_purposes"].value_counts()
df_unique_purp_count

недвижимость          7649
автомобиль            4306
образование           4013
цель не определена    3162
свадьба               2324
Name: lemmas_purposes, dtype: int64

**Вывод**

Наиболее часто клиенты банка берут кредит для операций, связанных с недвижимостью. Количество обращений по причине приобретения автомобилей и получения образования примерно одиковое и значительно меньше. Реже всего клиенты банка обращаются за кредитом для свадебных мероприятий.

### Категоризация данных

Создадим "словарь", где каждой из 21454 статистической записи обращений клиетов запишем 'education_id' и 'education'.

In [24]:
# создание отдельного «словаря», где названию категории 'education' соответствует номер 'education_id'
rest_education = df[['education_id','education']]
rest_education.head(5)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


В "словаре" присутствуют дубликаты. Их нужно удалить. Применим цепочку методов: drop_duplicates() и reset_index().

In [25]:
# удаление дубликатов (с удалением старых индексов и формированием новых)
rest_education = rest_education.drop_duplicates().reset_index(drop=True)
rest_education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Аналогично создадим "словать" для столбцов 'family_status_id', 'family_status' и для 'income_type' и удалим дубликаты.

In [26]:
# создание отдельного «словаря», где названию категории 'family_status' соответствует номер 'family_status_id'
rest_family = df[['family_status_id','family_status']]
# удаление дубликатов (с удалением старых индексов и формированием новых)
rest_family = rest_family.drop_duplicates().reset_index(drop=True)
rest_family

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [27]:
# создание отдельного «словаря» категории 'income_type'
rest_type = df[['income_type']]
# удаление дубликатов (с удалением старых индексов и формированием новых)
rest_type = rest_type.drop_duplicates().reset_index(drop=True)
rest_type

,income_type
0,сотрудник
1,пенсионер
2,компаньон
3,госслужащий
4,безработный
5,предприниматель
6,студент
7,в декрете


Проанализруем содержимое столбца 'gender'.

In [28]:
# создание отдельного «словаря» категории 'gender'
rest_gender = df[['gender']]
# удаление дубликатов (с удалением старых индексов и формированием новых)
rest_gender = rest_gender.drop_duplicates().reset_index(drop=True)
rest_gender

,gender
0,F
1,M
2,XNA


In [29]:
# просмотр уникальных значений и их количества столбца 'gender'
df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

При категоризации столбца 'gender' обнаружена запись третьего вида пола, что, скорее всего, является неправильным соединением данных из разных источников или ошибкой сотрудника при занесении информации. Воспользуемся методом loc для замены категориального значения. Большая часть пользователей банка являются женщины, поэтому с большей вероятностью можем заменить значение 'XNA' на 'F' в столбце 'gender'.

In [30]:
# замена строковых значений методом loc с осуществлением логической индексации
df.loc[df['gender'] == 'XNA','gender'] = 'F'
# просмотр уникальных значений и их количества столбца 'gender'
df['gender'].value_counts()

F    14175
M     7279
Name: gender, dtype: int64

In [31]:
# просмотр уникальных значений и их количества столбца 'children'
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение 20 в столбце 'children' аномально большое, можем предположить, что имеет место ошибка в данных. Вместо 20 детей принимаем 2 ребенка. Также заменим колчество детей -1 на 1.

In [32]:
# замена строковых значений методом loc с осуществлением логической индексации
df.loc[df['children'] == 20,'children'] = 2
# просмотр уникальных значений и их количества столбца 'gender'
df['children'].value_counts()

 0    14091
 1     4808
 2     2128
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

In [33]:
# замена строковых значений методом loc с осуществлением логической индексации
df.loc[df['children'] == -1,'children'] = 1
# просмотр уникальных значений и их количества столбца 'gender'
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Узнаем разброс значений, среднее значение и медиану, используя метод describe к столбцу 'total_income'.'education'.

In [34]:
# использование метода describe к столбцу'total_income'
df['total_income'].describe()

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Создадим функцию для категоризации уровня доходов клинтов банка, где на вход функции попадает ежемесячный доход, а возвращает она категорию клиента.

In [35]:
# создание функции total_income_group для категоризации уровня доходов клинтов банка
def total_income_group(total_income):
    if total_income <= 40000:
        return 'низкий уровень дохода'
    if total_income <= 80000:
        return 'средний уровень дохода'
    if total_income <= 160000:
        return 'высокий уровень дохода'
    return 'очень высокий уровень дохода'

Создадим отдельный столбец с категориями уровня доходов, и в его ячейках запишем значения, возвращаемые функцией total_income_group.

In [36]:
# создание отдельного столбца с категориями уровня доходов
df['total_income_group'] = df['total_income'].apply(total_income_group)

In [37]:
# проверка результатов
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purposes,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,цель не определена,очень высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,высокий уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,цель не определена,высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,очень высокий уровень дохода
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий уровень дохода


**Вывод**

Проведена категоризация данных и выялены все категориальные переменные. Также при категоризации были выялены ошибки и учтены в окончательных результатх.

## Анализ факторов, влияющих на возврата кредита в срок

### Анализ корреляции наличия детей и возврата кредита в срок

Сделаем сводную таблицу методом pivot_table().

In [38]:
# делаем сводную таблицу. Столбец, по которому группируем данные - children. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_children = df.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_children['mean'] = pivot_children['mean'] * 100
# именуем столбцы
pivot_children.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_children = pivot_children.sort_values(by = '%', ascending = False)
# округляем проценты в столбце '%' таблицы pivot_children до двух знаков после запятой
pivot_children['%'] = pivot_children['%'].round(2)
# выводим
display(pivot_children)


,has_debt,total,%
children,,,
4,4,41,9.76
2,202,2128,9.49
1,445,4855,9.17
3,27,330,8.18
0,1063,14091,7.54
5,0,9,0.00


**Вывод**

По представленным результатам можно сделать вывод, что на задолжность по возврату влияет скорее наличие детей, чем их количество. При этом колличество семей с пятью детьми существенно меньше и принимать их к оценке некорректно. В остальных случаях  доля клиентов (семьи с детьми), имеющих задолжность по возврату, приблизительно равна.

### Анализ корреляции семейного положения и возврата кредита в срок

In [39]:
# делаем сводную таблицу. Столбец, по которому группируем данные - family_status. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_family_status = df.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_family_status['mean'] = pivot_family_status['mean'] * 100
# именуем столбцы
pivot_family_status.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_family_status = pivot_family_status.sort_values(by = '%', ascending = False)
# округляем проценты в столбце '%' таблицы pivot_family_status до двух знаков после запятой
pivot_family_status['%'] = pivot_family_status['%'].round(2)
# выводим
display(pivot_family_status)

,has_debt,total,%
family_status,,,
Не женат / не замужем,274,2810,9.75
гражданский брак,388,4151,9.35
женат / замужем,931,12339,7.55
в разводе,85,1195,7.11
вдовец / вдова,63,959,6.57


**Вывод**

Самый большой процент должников банка составляют клиенты, не состоящие в официальном браке и состоящие в гражданском браке. Самый маленький процент имеет категория клиентов "вдовец / вдова". 

### Анализ корреляции уровня дохода и возврата кредита в срок

In [40]:
# делаем сводную таблицу. Столбец, по которому группируем данные - total_income_group. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_total_income_group = df.pivot_table(index = ['total_income_group'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_total_income_group['mean'] = pivot_total_income_group['mean'] * 100
# именуем столбцы
pivot_total_income_group.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_total_income_group = pivot_total_income_group.sort_values(by = '%', ascending = False)
# округляем проценты в столбце '%' таблицы pivot_total_income_group до двух знаков после запятой
pivot_total_income_group['%'] = pivot_total_income_group['%'].round(2)
# выводим
display(pivot_total_income_group)

,has_debt,total,%
total_income_group,,,
высокий уровень дохода,899,10458,8.60
низкий уровень дохода,10,125,8.00
очень высокий уровень дохода,668,8720,7.66
средний уровень дохода,164,2151,7.62


**Вывод**

Качественно уровень дохода не влияет на возврат кредита в срок.

### Анализ влияния целей кредита на его возврат в срок

In [41]:
# делаем сводную таблицу. Столбец, по которому группируем данные - lemmas_purposes. Значения для сводной таблицы по колонке 
# debt
# применяем функции sum, count, mean к значениям.
pivot_purposes = df.pivot_table(index = ['lemmas_purposes'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# отражаем процент
pivot_purposes['mean'] = pivot_purposes['mean'] * 100
# именуем столбцы
pivot_purposes.columns = ['has_debt', 'total', '%']
# сортируем таблицу по убыванию
pivot_purposes = pivot_purposes.sort_values(by = '%', ascending = False)
# округляем проценты в столбце '%' таблицы pivot_purposes до двух знаков после запятой
pivot_purposes['%'] = pivot_purposes['%'].round(2)
# выводим
display(pivot_purposes)


,has_debt,total,%
lemmas_purposes,,,
автомобиль,403,4306,9.36
образование,370,4013,9.22
свадьба,186,2324,8.00
недвижимость,568,7649,7.43
цель не определена,214,3162,6.77


**Вывод**

Самый низкий процент не возврата кредита в срок характерен для клиентов с целью покупки недвижимости и организации свадебных мероприятий. Процент должников по кредиту с целью покупки автомобиля и получения образования выше, такие займы более рисованы для банка. 

## Общий вывод

На процент возврата кредита влияют такие параметры как наличие детей, семейное положение, цель получения кредита. На процент возврата кредита не влияет уровень дохода.
Самые надежные клиенты без детей, не женатые, с целью кредита - покупка недвижимости. Самые ненадежные клиенты, имеющие детей, не состоящие в официальном браке, с целью получения кредита - покупка автомобиля.  